# What's in the notebook
- [Build an RNN from Scratch using Numpy](#scratch)

Recurrent Neural Networks (RNN) are effective for sequence tasks such as natural language processing and time-series forecasting. RNNs can read inputs $x^{<t>}$ one at a time and remember some information or context through the hidden layer activations that get passed from one time-step to the next. A uni-directional RNN takes information from the past to process future inputs. A bi-directional RNN can take context from both the past and the future. 

Notation used in this notebook:
- Layers:
    - $a^{[l]}$: activation of the $l$-th layer
    - $a^{[l]}_i$: the $i$-th activation unit in layer $l$
    - $a^{<t>}$: activation at the $t$-th timestep
    - $W^{[l]}$: weights of the $l$-th layer
    - $b^{[l]}$: bias of the $l$-th layer
- Inputs:
    - $x^{(i)}$: the $i$-th training example
    - $x^{<t>}$: the input x at the $t$-th timestep
    - $x^{(i)<t>}$: the input at the $t$-th timestep of the $i$-th example    

## <a id='scratch'></a>Build an RNN from Scratch using Numpy
The basic strucuture of a simple RNN with basic RNN cells is as below:
<img src="images/rnn (1).png" style="width:300;height:300px;">
<caption><center> **Figure 1**: Basic RNN model. *Source*: deeplearning.ai</center></caption>

**A basic RNN-cell**:
<img src="images/rnn_step_forward.png" style="width:700px;height:300px;">
<caption><center> **Figure 2**: Basic RNN cell. *Source*: deeplearning.ai </center></caption>
This RNN cell takes as input $x^{\langle t \rangle}$ (current input) and $a^{\langle t - 1\rangle}$ (previous hidden state containing information from the past), and outputs $a^{\langle t \rangle}$ which is given to the next RNN cell and also used to predict $y^{\langle t \rangle}$.


**Backpropagation**:
<img src="images/rnn_cell_backprop.png" style="width:500;height:300px;"> <br>
<caption><center> **Figure 4**: RNN-cell's backward pass. *Source*: deeplearning.ai </center></caption>

In [2]:
import numpy as np

# helper functions
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def initialize_adam(parameters) :
    '''
    Initializes v and s as two python dictionaries with:
                - keys: 'dW1', 'db1', ..., 'dWL', 'dbL' 
                - values: numpy arrays of zeros of the same shape as the corresponding gradients/parameters
    
    :params parameters: python dictionary containing parameters
                - parameters['W' + str(l)] = Wl
                - parameters['b' + str(l)] = bl
    
    :return:
    v: python dictionary that contains the exponentially weighted average of the gradient
    s: python dictionary that contains the exponentially weighted average of the squared gradient
    '''
    
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    s = {}
    
    # Initialize v, s. Input: 'parameters'. Outputs: 'v, s'.
    for l in range(L):
        v['dW' + str(l+1)] = np.zeros(parameters['W' + str(l+1)].shape)
        v['db' + str(l+1)] = np.zeros(parameters['b' + str(l+1)].shape)
        s['dW' + str(l+1)] = np.zeros(parameters['W' + str(l+1)].shape)
        s['db' + str(l+1)] = np.zeros(parameters['b' + str(l+1)].shape)

    return v, s


def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
    '''
    Update parameters using Adam
    
    :params parameters: python dictionary containing parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    :params grads: python dictionary containing gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    :params v: Adam variable, moving average of the first gradient, python dictionary
    :params s: Adam variable, moving average of the squared gradient, python dictionary
    :params learning_rate: the learning rate, scalar.
    :params beta1: Exponential decay hyperparameter for the first moment estimates 
    :params beta2: Exponential decay hyperparameter for the second moment estimates 
    :params epsilon: hyperparameter preventing division by zero in Adam updates

    :return:
    parameters: python dictionary containing your updated parameters 
    v: Adam variable, moving average of the first gradient, python dictionary
    s: Adam variable, moving average of the squared gradient, python dictionary
    '''
    
    L = len(parameters) // 2                 # number of layers in the neural networks
    v_corrected = {}                         # Initializing first moment estimate, python dictionary
    s_corrected = {}                         # Initializing second moment estimate, python dictionary
    
    # Adam update on all parameters
    for l in range(L):
        # Moving average of the gradients 
        v['dW' + str(l+1)] = beta1 * v['dW' + str(l+1)] + (1 - beta1) * grads['dW' + str(l+1)] 
        v['db' + str(l+1)] = beta1 * v['db' + str(l+1)] + (1 - beta1) * grads['db' + str(l+1)] 

        # Compute bias-corrected first moment estimate
        v_corrected['dW' + str(l+1)] = v['dW' + str(l+1)] / (1 - beta1**t)
        v_corrected['db' + str(l+1)] = v['db' + str(l+1)] / (1 - beta1**t)

        # Moving average of the squared gradients
        s['dW' + str(l+1)] = beta2 * s['dW' + str(l+1)] + (1 - beta2) * (grads['dW' + str(l+1)] ** 2)
        s['db' + str(l+1)] = beta2 * s['db' + str(l+1)] + (1 - beta2) * (grads['db' + str(l+1)] ** 2)

        # Compute bias-corrected second raw moment estimate
        s_corrected['dW' + str(l+1)] = s['dW' + str(l+1)] / (1 - beta2 ** t)
        s_corrected['db' + str(l+1)] = s['db' + str(l+1)] / (1 - beta2 ** t)

        # Update parameters
        parameters['W' + str(l+1)] = parameters['W' + str(l+1)] - learning_rate * v_corrected['dW' + str(l+1)] / np.sqrt(s_corrected['dW' + str(l+1)] + epsilon)
        parameters['b' + str(l+1)] = parameters['b' + str(l+1)] - learning_rate * v_corrected['db' + str(l+1)] / np.sqrt(s_corrected['db' + str(l+1)] + epsilon)

    return parameters, v, s

In [4]:
# forward step
def rnn_cell_forward(xt, a_prev, parameters):
    '''
    Implements a single forward step of the basic RNN-cell described in Figure 2
    
    :params xt: numpy array of shape (n_x, m) representing input at timestep t
    :params a_prev: numpy array of shape (n_a, m) representing hidden state at timestep t-1
    :params parameters: python dictionary containing layer parameters:
                        {Wax: numpy array of shape (n_a, n_x) representing the weight matrix multiplying the input xt,
                         Waa: numpy array of shape (n_a, n_a) representing the weight matrix multiplying the hidden state a_prev,
                         Wya: numpy array of shape (n_y, n_a) representing the weight matrix relating to the hidden state to the output,
                         ba:  numpy array of shape (n_a, 1) representing the bias,
                         by:  numpy array of shape (n_y, 1) representing the bias relating the hidden-state to the output}

    :return: 
    a_next: next hidden state of shape (n_a, m)
    yt_pred: prediction at timestep t of shape (n_y, m)
    cache: tuple of values (a_next, a_prev, xt, parameters) needed for the backward pass
    '''
    Wax = parameters['Wax']
    Waa = parameters['Waa']
    Wya = parameters['Wya']
    ba = parameters['ba']
    by = parameters['by']
    
    # compute the hidden state a<t> with tanh activation
    a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, xt) + ba)
    
    # compute the prediction ŷ⟨t⟩
    yt_pred = softmax(np.dot(Wya, a_next) + by)
    
    # store values you need for backward propagation in cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

In [5]:
def rnn_forward(x, a0, parameters):
    '''
    Implement the forward propagation of the recurrent neural network.

    :params x: Input data for every time-step, of shape (n_x, m, T_x)
    :params a0: Initial hidden state, of shape (n_a, m)
    :params parameters: python dictionary containing layer parameters:
                        {Wax: numpy array of shape (n_a, n_x) representing the weight matrix multiplying the input xt,
                         Waa: numpy array of shape (n_a, n_a) representing the weight matrix multiplying the hidden state a_prev,
                         Wya: numpy array of shape (n_y, n_a) representing the weight matrix relating to the hidden state to the output,
                         ba:  numpy array of shape (n_a, 1) representing the bias,
                         by:  numpy array of shape (n_y, 1) representing the bias relating the hidden-state to the output}

    :return:
    a: Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y_pred: Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches: tuple of values needed for the backward pass, contains (list of caches, x)
    '''
    # Initialize 'caches' which will contain the list of all caches
    caches = []
    
    # Retrieve dimensions from shapes of x and parameters['Wya']
    n_x, m, T_x = x.shape
    n_y, n_a = parameters['Wya'].shape
    
    # initialize a and y with zeros
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    
    # Initialize a_next
    a_next = a0
    
    # loop over all time-steps
    for t in range(T_x):
        # Update next hidden state, compute the prediction, get the cache
        a_next, yt_pred, cache = rnn_cell_forward(x[:, :, t], a_next, parameters)
        # Save the value of the new next hidden state in a
        a[:,:,t] = a_next
        # Save the value of the prediction in y
        y_pred[:,:,t] = yt_pred
        # Append cache to caches 
        caches.append(cache)
    
    # store values needed for backward propagation in cache
    caches = (caches, x)
    
    return a, y_pred, caches

In [6]:
def rnn_cell_backward(da_next, cache):
    '''
    Implements the backward pass for the RNN-cell (single time-step) in Figure 3.

    :params da_next: Gradient of loss with respect to next hidden state
    :params cache: python dictionary containing useful values (output of rnn_cell_forward())

    :return:
    gradients: python dictionary containing:
                {dx: Gradients of input data, of shape (n_x, m),
                 da_prev: Gradients of previous hidden state, of shape (n_a, m),
                 dWax: Gradients of input-to-hidden weights, of shape (n_a, n_x),
                 dWaa: Gradients of hidden-to-hidden weights, of shape (n_a, n_a),
                 dba: Gradients of bias vector, of shape (n_a, 1)}
    '''
    # Retrieve values from cache
    (a_next, a_prev, xt, parameters) = cache
    
    # Retrieve values from parameters
    Wax = parameters['Wax']  # (n_a, n_x)
    Waa = parameters['Waa']  # (n_a, n_a)
    Wya = parameters['Wya']  # (n_y, n_a)
    ba = parameters['ba']    # (n_a, 1)
    by = parameters['by']    # (n_y, 1)

    # compute the gradient of tanh with respect to a_next
    dtanh = da_next * (1. - np.square(np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + ba))) # (n_a, m)

    # compute the gradient of the loss with respect to Wax
    dxt = np.dot(Wax.T, dtanh)
    dWax = np.dot(dtanh, xt.T)

    # compute the gradient with respect to Waa
    da_prev = np.dot(Waa.T, dtanh)
    dWaa = np.dot(dtanh, a_prev.T)

    # compute the gradient with respect to b
    dba = np.sum(dtanh, axis=1, keepdims=True)

    # Store the gradients in a python dictionary
    gradients = {"dxt": dxt, "da_prev": da_prev, "dWax": dWax, "dWaa": dWaa, "dba": dba}
    
    return gradients

In [7]:
def rnn_backward(da, caches):
    '''
    Implement the backward pass for a RNN over an entire sequence of input data.
   
    :params da: Upstream gradients of all hidden states, of shape (n_a, m, T_x)
    :params caches: tuple containing information from the forward pass (rnn_forward)
    
    :return:
    gradients: python dictionary containing:
            {dx: Gradients of input data, of shape (n_x, m),
             da0: Gradients of initial hidden state, of shape (n_a, m),
             dWax: Gradients of input-to-hidden weights, of shape (n_a, n_x),
             dWaa: Gradients of hidden-to-hidden weights, of shape (n_a, n_a),
             dba: Gradients of bias vector, of shape (n_a, 1)}
    '''
    # Retrieve values from the first cache (t=1) of caches 
    (caches, x) = caches
    (a1, a0, x1, parameters) = caches[0]
    
    # Retrieve dimensions from da's and x1's shapes 
    n_a, m, T_x = da.shape
    n_x, m = x1.shape
    
    # initialize the gradients
    dx = np.zeros((n_x, m, T_x))
    dWax = np.zeros((n_a, n_x))
    dWaa = np.zeros((n_a, n_a))
    dba = np.zeros((n_a, 1))
    da0 = np.zeros((n_a, m))
    da_prevt = np.zeros((n_a, m))
    
    # Loop through all the time steps
    for t in reversed(range(T_x)):
        # Compute gradients at time step t.
        gradients = rnn_cell_backward(da[:, :, t] + da_prevt, caches[t])
        # Retrieve derivatives from gradients
        dxt, da_prevt, dWaxt, dWaat, dbat = gradients['dxt'], gradients['da_prev'], gradients['dWax'], gradients['dWaa'], gradients['dba']
        # Increment global derivatives w.r.t parameters by adding their derivative at time-step t
        dx[:, :, t] = dxt
        dWax += dWaxt
        dWaa += dWaat
        dba += dbat
        
    # Set da0 to the gradient of a which has been backpropagated through all time-steps 
    da0 = da_prevt

    # Store the gradients in a python dictionary
    gradients = {"dx": dx, "da0": da0, "dWax": dWax, "dWaa": dWaa,"dba": dba}
    
    return gradients